In [1]:
#IMPORTS
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import preprocessing

## Part 3: VALIDATOR to test
modifier and simulator working in dynamic way, with hardcoded ranges
* next step, extract the ranges 



In [2]:
# DRAFT TEMP FUNCTIONS that will be in validator once they work as intended
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import PolynomialFeatures

def get_range(x_values,outliers,delta=0.7):
    x_values = np.array(x_values)
    relative_max = x_values[-1]
    relative_min = x_values[0]
    
    # Calculate ranges of outliers
    outlier_ranges = []

    current_range = [outliers[0]]
    print('current range: ',current_range)

    for i in range(1, len(outliers)):
        x_outlier_ranges = []

        if outliers[i] == outliers[i-1] + 1:
            current_range.append(outliers[i])
        else:
            
            outlier_ranges.append(current_range)
            current_range = [outliers[i]]

    # Add the last range
    outlier_ranges.append(current_range)
    print(outlier_ranges)
    # Get x value ranges for each outlier range
    x_range_max=relative_max
    x_range_min = relative_min
    for outlier_range in outlier_ranges:
        print('\noutlier_range ',outlier_range)
        if np.max(outlier_range)<(np.shape(x_values)[0]-1) and np.min(outlier_range)>0: 
            delta_low=delta*(x_values[np.min(outlier_range)] - x_values[np.min(outlier_range)-1])
            delta_high=delta*np.abs(x_values[np.max(outlier_range)] - x_values[np.max(outlier_range)+1])
            x_range_min = np.min(x_values[outlier_range])-delta_low
            x_range_max = np.max(x_values[outlier_range])+delta_high
            print('I am withing the min max')
        elif x_range_max>relative_max or np.max(outlier_range)==(np.shape(x_values)[0]-1):
            print('I am at max')
            delta_low=delta*(x_values[np.min(outlier_range)] - x_values[np.min(outlier_range)-1])
            x_range_min = np.min(x_values[outlier_range])-delta_low
            x_range_max=relative_max
        elif x_range_min<relative_min or np.min(outlier_range)==0:
            print('I am at min')
            delta_high=delta*np.abs(x_values[np.max(outlier_range)] - x_values[np.max(outlier_range)+1])
            x_range_max = np.max(x_values[outlier_range])+delta_high
            x_range_min = relative_min
        x_outlier_ranges.append((x_range_min, x_range_max))


    print("Ranges of outlier x values:")
    for i, x_range in enumerate(x_outlier_ranges):
        print(f"Outlier Range {i+1}: {x_range[0]} to {x_range[1]}")
    
    return x_outlier_ranges



def curve_fit(mod_x,sim_y, threshold=3):
    # Example data
    # mod_x = [1.0, 2.4347826086956523, 6.260869565217392, 12.478260869565217, 21.08695652173913, 32.086956521739125, 45.47826086956522, 61.26086956521739, 79.43478260869566, 100.0]
    # sim_y = [-18785.46448566434, -22589.28977083527, -13875.22366018544, -8366.169916783167, -10461.393801292468, 5188.58062260236, 76235.86972207189, 197523.09425923007, 435446.98046035116, 1004675.2544642929]

    # Reshape the arrays to match HuberRegressor's expectations
    x_values = np.array(mod_x)
    y_values = np.array(sim_y)

    # Create polynomial features
    poly = PolynomialFeatures(degree=2)
    x_poly = poly.fit_transform(x_values.reshape(-1, 1))

    # Fit a polynomial model using HuberRegressor
    huber_regressor = HuberRegressor()
    huber_regressor.fit(x_poly, y_values)

    # Predict y values based on the fitted polynomial model
    predicted_y_values = huber_regressor.predict(x_poly)

    # Calculate residuals (differences between actual y and predicted y)
    residuals = y_values - predicted_y_values

    # Calculate the median absolute deviation (MAD) of the residuals
    median_absolute_deviation = np.median(np.abs(residuals))

    # Define a threshold for outliers (e.g., 3 times MAD)
    outlier_threshold = threshold * median_absolute_deviation

    # Identify outliers
    outliers = np.where(np.abs(residuals) > outlier_threshold)[0]

    # Identify points of good fit
    good_fit_points = np.where(np.abs(residuals) <= outlier_threshold)[0]

    print("Outlier indices:", outliers)
    print("Good fit indices:", good_fit_points)

    # Plot the original data and the fitted polynomial curve
    plt.scatter(x_values, y_values, label='Data points')
    plt.plot(x_values, predicted_y_values, color='red', label='Fitted polynomial')
    plt.legend()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Fitted Polynomial and Data Points')
    plt.show()
    return outliers, good_fit_points


## PART 4: Main Func: with Modifier.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
from global_settings import fitting_threshold, mdv


# import global here: threshold

def __init__(self): #i.e. if first iteration
    self.iterations = 1
    self.total_points = 0
    self.total_bad_points = 0
    self.range = (mdv["domain_min_range"],mdv["domain_max_range"])  # Initialize the range as a tuple
    self.num_points_evaluated = 0  # Initialize num_points_evaluated


def fit_curve(x_values,y_values,global_range):
    # Assuming you have a function to fit a curve to the data
    # Replace the placeholder code below with your curve fitting logic

    # standardize    
    x_scaler, y_scaler = StandardScaler(), StandardScaler()
    x_train = x_scaler.fit_transform(x_values)
    y_train = y_scaler.fit_transform(y_values)

    # fit model
    model = HuberRegressor(epsilon=1)
    model.fit(x_train, y_train.ravel())

    # Fit curve to the data
    fitted_curve = np.polyfit(x_values, y_values, 1)
    return fitted_curve

def get_unfitting_point(x_values, y_values,fitted_curve,threshold = 0.9):
    #extract points that are farther than the threshold
    # label them as unfitting points
    temp_soln = [1,4,5,6] 
    unfit_points = [x_values[temp_soln],y_values[temp_soln]]
    return unfit_points

def get_unfitting_ranges(x_values,sim_y_list,threshold=fitting_threshold):
    # apply curve fit to new data
    fitted_curve = Validator.fit_curve(x_values,sim_y_list)
    # get points of unfit
    unfit_points, fit_points = Validator.thresholding(fitted_curve,threshold)

    # create ranges from continuous unfit points
    if Validator.iterations == 0:
        unfit_ranges = [[0,4],[40,50]]

    Validator.collect_data(sim_y_list,x_values,unfit_ranges,unfit_points=unfit_points)
    # return unfit ranges
    Validator.collect_data()
    return unfit_ranges

def update_history(self, new_total_points, new_range):
    self.iterations += 1
    self.total_points += new_total_points
    self.range = new_range

def update_num_points_evaluated(self, points, min_x, max_x):
    self.num_points_evaluated = len([(x, y) for x, y in points if min_x <= x <= max_x])

def local_exploration_validator_A(x_values, y_values, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=fitting_threshold):
    # generate fit function 
    fitted_curve = Validator.fit_curve(x_values, y_values,global_range)
    # find misfit points from mod_x_list and sim_y_list (outliers) using threshold from fit function
    unfitting_points = Validator.get_unfitting_point(x_values, y_values,fitted_curve,threshold = 0.9)
    # generate ranges of misfit points (make it fancy threshold)
    unfitting_ranges = Validator.get_unfitting_ranges(unfitting_points)
    ## update your history (for each iteration: 

    # Update history with new values
    Validator.update_history(5, 100, (10, 20))  # For example, 5 new iterations, 100 new total points, and new range

    # Update num_points_evaluated (self, new_iterations, new_total_points, new_range)
    Validator.update_num_points_evaluated(len(x_values), min=global_range[0], max=global_range[1])

    # Print updated values
    print(f"Iterations: {Validator.iterations}")
    print(f"Total Points: {Validator.total_points}")
    print(f"Range: {Validator.range}")
    print(f"Num Points Evaluated: {Validator.num_points_evaluated}")

    # iteration, 
    # total points evaluated (good and misfit), 
    # points evaluated this iteration (good and misfit), 
    # number of misfit ranges)

    # num_points_input = len(mod_x_list)
    # total_points.append(mod_x_list)

    # # Get range from previous iteration range generation
    # min_x,max_x = range_iteration 
    # num_points_evaluated = [(x, y) for x, y in total_points if min_x <= x <= max_x]
    return unfitting_ranges


def validator_controller(mod_x_list,sim_y_list, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=fitting_threshold, local_validator=local_exploration_validator_A, do_plot=False):
    # gets points mod_x_list, sim_y_list
    validator_ranges=Validator.local_exploration_validator_A(mod_x_list,sim_y_list, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=fitting_threshold)

    # if not first time accessing validator, merge old points with new
        # i.e. merge all data points together
    return validator_ranges


def collect_data(self, sym, mod, ranges, unfit_points):
    self.archive_sym.append(sym)
    self.archive_mod.append(mod)
    # track number of iterations
    self.iterations = self.iterations+1
    self.total_points = self.total_points+len(mod)
    # track number of good/bad points
    self.total_unfit_points = self.total_unfit_points+len(mod)

def update_statistics(self, new_sym, new_mod):
    # track number of points generated from input to get_unfitting_ranges
    self.collect_data(new_sym,new_mod)
    # track number of unfit intervals i.e. append length of ranges each itteration)
    self.history.extend(new_sym)



def thresholding(self, threshold):
    # Assuming you want to return x values above the threshold
    x_values = np.arange(len(self.history))
    y_values = np.array(self.history)
    above_threshold = x_values[y_values > threshold]
    return above_threshold


# # Example usage
# analyzer = Validator()
# data = [1, 2, 3, 4]
# analyzer.collect_data(data)
# new_data = [5, 6, 7]
# analyzer.update_history(new_data)
# print(analyzer.history)  # Output: [1, 2, 3, 4, 5, 6, 7]

# fitted_curve = analyzer.fit_curve()
# print(fitted_curve)  # Output: [slope, intercept]

# threshold = 3
# above_threshold = analyzer.thresholding(threshold)
# print(above_threshold)  # Output: [3, 4, 5, 6]


In [3]:
# PLACEHOLDER FOR THE MAIN FUNCTION V2
%load_ext autoreload
%autoreload 2

from global_settings import mdv
from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator

print(mdv)
#TODO: How to autoreload without restarting kernel (for global vars)
# Instantiate mdv values
# mdv={"domain_min_range":1, "domain_max_range":100, "modifier_incremental_unit":3, "modifier_data_point":10}
# Create an instance of the Validator Class
validate = Validator
modifier = Modifier
simulator = Simulator
appendedvars=[]

mainfunc=True
# Initialize interval list
interval_lists=[(mdv["domain_min_range"], mdv["domain_max_range"])]
x=1
while mainfunc==True:

    # Run Modifier Controller Function
    mod_x_list= modifier.modifier_controller(range_list=interval_lists,local_modifier=modifier.local_modifier_A, do_plot=False)
    # appendedvars.append(mod_x_list)

    #if data can't be generated:
    if mod_x_list == False: # FALSE IF ["modifier_data_point"] < mdv["modifier_incremental_unit"]:
        print('*   ITERATIONS END HERE   *')
        break

    # Run Simulator Function
    
    # for i in range(np.shape(mod_x_list)[0]):

    mod_x,sim_y_list = simulator.simulator_controller(mod_x_list,selected_function=simulator.sim_func_A)
    # print((mod_x))
    # print((sim_y_list))
    assert len(mod_x) == len(sim_y_list)
    # Get ranges of unfit points ( IF ANY )

    # Run Validator Controller functions
    output = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
    
        
    # Example Usage:

    # TODO fix error in Validator function
    # TODO: Generate Report
    # ranges = validate.get_unfitting_ranges(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
    # print('these are the ranges of validator\n',ranges)

    ## TODO: Replace for loops with Validator functions that return ranges [(),(),()] format
    if x == 1: 
        ranges = [(0,50),(70,80)]
    elif x == 2:
        ranges = [(30,50),(70,75)]
    elif x==3: 
        ranges = False
    x=x+1
    if mod_x_list == False or not ranges: 
        mainfunc=False
        print('The END')
    interval_lists = ranges


# TODO: 
# Validator needs to work as a loop within itseft that accepts 3 different sets of points
# understand curve fitting
# Fix ranges


{'domain_min_range': 1, 'domain_max_range': 100, 'modifier_incremental_unit': 3, 'modifier_data_point': 10}

Modifier controller...
  * Interval:  [(1, 100)]
     * Iterations within Modifier:  0
  * Mod_x shape:    (1, 10)

Simulator...
  * Sim_y shape:    (10,)


ValueError: Expected 2D array, got 1D array instead:
array=[  1.           2.43478261   6.26086957  12.47826087  21.08695652
  32.08695652  45.47826087  61.26086957  79.43478261 100.        ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.